# Conformer Generation Pipeline

## **Step 1: Input Smile and Reference Conformer**

In [ ]:
# import os
# # Specify which GPUs to use
# os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"
# os.environ["OMP_NUM_THREADS"] = "25"
# os.environ["MKL_NUM_THREADS"] = "25"
# os.environ["OPENBLAS_NUM_THREADS"] = "25"

In [ ]:
import time

# Capture the start time
start_time: float = time.time()

In [ ]:
import sys

# Add a folder to Python's import path
sys.path.append('/work') 

from atk_conformer_generation_pipeline.utils import *
from atk_conformer_generation_pipeline.variables import *

In [ ]:
#Change the dir to /work
os.chdir("/work")
!pwd

**Change the below variables accordingly**

In [ ]:
os.makedirs(output_dir, exist_ok=True)
os.chdir(output_dir)

In [ ]:
!pwd

In [ ]:
import sys
sys.setrecursionlimit(10000)
# set the maximum depth of the Python interpreter stack. This stack depth is crucial for recursive function calls, 
# as it limits how deep the recursion can go before causing a RecursionError.

In [ ]:
display_2d_structure(inp_smiles)


**Importing the necessary libraries**

In [ ]:
%%time

from GNNImplicitSolvent import minimize_mol, calculate_entropy
from rdkit import Chem
from rdkit.Chem import AllChem

mol = Chem.MolFromSmiles(inp_smiles)
mol = Chem.AddHs(mol)
AllChem.EmbedMultipleConfs(mol, numConfs=num_conf, useExpTorsionAnglePrefs = False)

minimized_mol, energies = minimize_mol(mol,Solvent)
#entropies, free_energies = calculate_entropy(minimized_mol,Solvent)

In [ ]:
import csv
# Convert to kcal/mol
energies_kcal = [e / 4.184 for e in energies]

# Save to CSV
with open('opt_conf_energies.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['conformer_id', 'energy_in_kcalpermol'])  # Header
    for i, energy in enumerate(energies_kcal):
        writer.writerow([i, energy])

In [ ]:
def save_conformers_to_sdf(mol: Chem.Mol, filename: str) -> None:
    """Save a molecule with multiple conformers to an SDF file.

    Args:
        mol (Chem.Mol): The molecule containing multiple conformers.
        filename (str): The name of the output SDF file.
    """
    writer = Chem.SDWriter(filename)
    
    for conf_id in range(mol.GetNumConformers()):
        mol.SetProp("_Name", f"conformer_{conf_id}")
        writer.write(mol, confId=conf_id)
    
    writer.close()

opt_conf_sdf="optimized_generated_conformers.sdf"
save_conformers_to_sdf(minimized_mol,opt_conf_sdf)